In [3]:
import os
import requests
import pandas as pd

import sys
sys.path.append('..')
from models.cases import Case

In [ ]:

API_ENDPOINT = "https://wcca.wicourts.gov/jsonPost/advancedCaseSearch"
START_DATE = "03-29-2024" ## MM-DD_YYYY FORMAT
END_DATE = "03-29-2024" ## MM-DD_YYYY FORMAT

headers = {
    'Content-Type': 'application/json'
}

payload = {
    "includeMissingDob":True,
    "includeMissingMiddleName":True,
    "attyType":"partyAtty",
    "filingDate":{"start":START_DATE,"end":END_DATE}
}

response = requests.post(
    url=API_ENDPOINT,
    json=payload,
    headers=headers
)

print(response.status_code)

In [2]:
cases = response.json().get("result").get("cases")

In [11]:
cases[:5]

[{'partyName': 'Schenck, Jerome E',
  'countyName': 'Dane',
  'dob': None,
  'caseNo': '2024WL000299',
  'countyNo': 13,
  'caption': 'In the Matter of Jerome E Schenck',
  'status': 'Filed Only',
  'filingDate': '2024-03-29'},
 {'partyName': 'Colbert, James Joseph',
  'countyName': 'Dane',
  'dob': None,
  'caseNo': '2024WL000298',
  'countyNo': 13,
  'caption': 'In the Matter of James Joseph Colbert',
  'status': 'Filed Only',
  'filingDate': '2024-03-29'},
 {'partyName': 'Colbert, James Joseph',
  'countyName': 'Dane',
  'dob': None,
  'caseNo': '2024WL000297',
  'countyNo': 13,
  'caption': 'In the Matter of James Joseph Colbert',
  'status': 'Filed Only',
  'filingDate': '2024-03-29'},
 {'partyName': 'Kollross, Nicole L',
  'countyName': 'Dane',
  'dob': None,
  'caseNo': '2024WL000296',
  'countyNo': 13,
  'caption': 'In the Matter of Nicole L Kollross',
  'status': 'Filed Only',
  'filingDate': '2024-03-29'},
 {'partyName': 'Meyers, Lorelie K',
  'countyName': 'Dane',
  'dob': N

In [4]:
df = pd.DataFrame(cases)

# Define the path to your CSV file
csv_file_temp_path = 'Temp/wisconsin.csv'

# Split the path in directory and file name
directory = os.path.dirname(csv_file_temp_path)

# Check if the directory exists, create it if it doesn't
if not os.path.exists(directory):
    os.makedirs(directory)

df.to_csv(csv_file_temp_path, index=False)

In [10]:
df.head(5)

,partyName,countyName,dob,caseNo,countyNo,caption,status,filingDate
0,"Schenck, Jerome E",Dane,None,2024WL000299,13,In the Matter of Jerome E Schenck,Filed Only,2024-03-29
1,"Colbert, James Joseph",Dane,None,2024WL000298,13,In the Matter of James Joseph Colbert,Filed Only,2024-03-29
2,"Colbert, James Joseph",Dane,None,2024WL000297,13,In the Matter of James Joseph Colbert,Filed Only,2024-03-29
3,"Kollross, Nicole L",Dane,None,2024WL000296,13,In the Matter of Nicole L Kollross,Filed Only,2024-03-29
4,"Meyers, Lorelie K",Dane,None,2024WL000295,13,In the Matter of Lorelie K Meyers,Filed Only,2024-03-29


Get Cookie to solve hcaptcha
To Do:
Solve the hcaptcha manually

In [5]:
from playwright.async_api import async_playwright

pw = await async_playwright().start()
browser = await pw.chromium.launch(headless=False)
context = await browser.new_context()
page = await context.new_page()

await page.goto(url="https://wcca.wicourts.gov/caseDetail.html?caseNo=2024TR004754&countyNo=13&index=0&isAdvanced=true&mode=details")


<Response url='https://wcca.wicourts.gov/caseDetail.html?caseNo=2024TR004754&countyNo=13&index=0&isAdvanced=true&mode=details' request=<Request url='https://wcca.wicourts.gov/caseDetail.html?caseNo=2024TR004754&countyNo=13&index=0&isAdvanced=true&mode=details' method='GET'>>

Get the Cookie After solving the manual captcha

In [6]:
async def get_cookies():
    await page.reload(wait_until="networkidle")
    await page.wait_for_selector('div.caseDetailContainer', state='attached', timeout=999999999)
    cookies = await page.context.cookies()
    cookies = '; '.join([f"{cookie.get('name')}={cookie.get('value')}" for cookie in cookies])
    return cookies

In [7]:
def get_case_details(caseNo, countyNo, cookies):
    headers = {
        'Content-Type': 'application/json',
        'Cookie': cookies
    }

    res = requests.post(
        url=f"https://wcca.wicourts.gov/caseDetail/{countyNo}/{caseNo}",
        headers=headers,
        json={}
    )

    if res.status_code == 200:
        return res.json()
    else:
        return None

In [12]:
print(Case.__fields__.keys())

dict_keys(['case_id', 'court_id', 'flag', 'urgent', 'participants', 'related_cases', 'protection_order', 'parties', 'disposed', 'legal_fileaccepted', 'paper_accepted', 'confidential', 'display_judgenotes', 'case_notecount', 'display_legalfileviewer', 'display_fileviewer', 'can_userseepublicdocuments', 'can_userseecasedocuments', 'can_userseeenoticehistory', 'can_selectdocket', 'can_seeecflinks', 'can_seelegalfilelinks', 'is_ticket', 'address_a_type', 'address_city', 'address_line_1', 'address_seq_no', 'address_state_code', 'address_zip', 'birth_date', 'birth_date_code', 'criminal_case', 'criminal_ind', 'description', 'description_code', 'first_name', 'year_of_birth', 'formatted_party_address', 'formatted_party_name', 'formatted_telephone', 'last_name', 'lit_ind', 'middle_name', 'party_type', 'pidm', 'pred_code', 'prosecuting_atty', 'pty_seq_no', 'sort_seq', 'age', 'case_desc', 'court_desc', 'location', 'filing_date', 'case_date', 'formatted_filingdate', 'case_type', 'case_security', 'c

In [9]:
from tqdm import tqdm

cookies = ""

details = []

for i, case in tqdm(enumerate(cases)):
    caseNo = case.get("caseNo")
    countyNo = case.get("countyNo")

    case_details = get_case_details(caseNo, countyNo, cookies)
    if case_details and case_details.get("errors"):
        print(i)
        cookies = await get_cookies()
        case_details = get_case_details(caseNo, countyNo, cookies)
    
    try:
        details.append({
            
        })
    except Exception as e:
        print(e)
        details.append({})
    
    # print(case_details)


0
8
107
206
305
404
503
602
701
800
899
